In [ ]:
!pip install litellm python-dotenv

In [1]:
import litellm
import os
from dotenv import load_dotenv
import sys
import requests

In [7]:
# Load environment variables
load_dotenv()

# Get values from environment
LITELLM_API_KEY = os.getenv("LITELLM_API_KEY")
LITELLM_BASE_URL = os.getenv("LITELLM_BASE_URL")

# Validate environment variables
if not LITELLM_API_KEY or not LITELLM_BASE_URL:
    print("Error: Missing API key or base URL in .env file")
    sys.exit(1)

litellm.api_base = LITELLM_BASE_URL
litellm.api_key = LITELLM_API_KEY

In [8]:
def get_available_models():
    """Get list of available models from your LiteLLM proxy"""
    
    headers = {
        "Authorization": f"Bearer {LITELLM_API_KEY}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(
            f"{LITELLM_BASE_URL}/models",
            headers=headers
        )
        
        if response.status_code == 200:
            models = response.json()
            return [model["id"] for model in models["data"]]
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def chat_with_model(model_name, message, stream=False):
    """Chat with any available model through LiteLLM proxy"""
    try:
        response = litellm.completion(
            model=model_name,
            messages=[{"role": "user", "content": message}],
            api_key=LITELLM_API_KEY,
            api_base=LITELLM_BASE_URL,
            stream=stream
        )
        
        if stream:
            full_response = ""
            for chunk in response:
                if chunk.choices[0].delta.content:
                    content = chunk.choices[0].delta.content
                    print(content, end="", flush=True)
                    full_response += content
            print()
            return full_response
        else:
            return response.choices[0].message.content
            
    except Exception as e:
        return f"Error: {str(e)}"

### Check available models

In [10]:
print("Available models:")
models = get_available_models()
if isinstance(models, list):
    for model in models[:10]:  # Show first 10 models
        print(f"- {model}")
else:
    print(models)

Available models:
- o1
- gpt-4.5-preview
- claude-haiku-3.5
- gpt-4.1-2025-04-14
- claude-sonet-3.7
- o3-mini-2025-01-31
- claude-opus-4
- claude-sonet-4
- gpt-4o-search-preview-2025-03-11
- openai/hd/1024-x-1024/dall-e-3


### Test basic chat

In [14]:
model_name = "anthropic/claude-haiku-3.5" # Check https://docs.litellm.ai/docs/providers for how to state model name depending on provider
message = "What goes well with pancakes besides jam?"
    
print(f"Testing with model: {model_name}")
print(f"Query: {message}")
    
result = chat_with_model(model_name, message)
print(f"Response: {result}")

Testing with model: anthropic/claude-haiku-3.5
Query: What goes well with pancakes besides jam?

Provider List: https://docs.litellm.ai/docs/providers

Response: Here are some delicious toppings and accompaniments for pancakes:

1. Sweet options:
- Maple syrup
- Honey
- Whipped cream
- Fresh berries
- Nutella
- Chocolate chips
- Caramel sauce
- Butter
- Powdered sugar

2. Fruit options:
- Sliced bananas
- Strawberries
- Blueberries
- Apple compote
- Peaches
- Raspberries

3. Savory options:
- Bacon
- Scrambled eggs
- Sausage
- Ham
- Cheese
- Fried chicken
- Smoked salmon

4. Spreads:
- Peanut butter
- Almond butter
- Cream cheese
- Yogurt

5. Nuts and seeds:
- Chopped almonds
- Pecans
- Walnuts
- Chia seeds
- Sunflower seeds

Each of these can add different flavors and textures to your pancakes, making them more interesting and enjoyable.
